<a href="https://colab.research.google.com/github/reeshabhkumarranjan/ML-Course-Project/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import numpy as np
import glob
from keras import Sequential
import sklearn, sklearn.model_selection
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Activation
from keras.utils import to_categorical, plot_model
from sklearn.svm import SVC
# from sklearn.externals import joblib
from joblib import dump, load

In [192]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

# loading data
colab_directory = '/content/drive/My Drive/Colab Notebooks/ML Course Project/'
x = np.load(colab_directory + "x.npy")
y = np.load(colab_directory + "y.npy")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
y_train_classes = y_train
y_test_classes = y_test
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# common variables/functions
epochs = 5
showWorking = False
showWorkingAverage = True
batchSize = 10
# dropoutRate = 0.5
activationFunction = 'relu'
iterations = 5
model_description_mode = False
iterations = 1 if model_description_mode else iterations
numFilters = 30

def evaluate_dl_model(model, x_train, x_test, y_train, y_test):
    accuracy_test = model.evaluate(x_test, y_test, batch_size=batchSize, verbose=showWorking)[1]
    accuracy_train = model.evaluate(x_train, y_train, batch_size=batchSize, verbose=showWorking)[1]

    # f1-score
    y_pred = model.predict_classes(x_test)
    # print(y_pred)
    f1_score = sklearn.metrics.f1_score(y_test_classes, y_pred, average='micro')
    
    return accuracy_train, accuracy_test, f1_score

def print_dl_metrics(accuracy_train, accuracy_test, f1_score, i, average=False):
    if not average:
        print(str(i + 1) + " train: " + str(accuracy_train / (i + 1)))
        print(str(i + 1) + " test : " + str(accuracy_test / (i + 1)))
        print(str(i + 1) + " f1   : " + str(f1_score / (i + 1)))
        print()
    else:
        print("Average Accuracy of LSTM model on train: ", accuracy_train / (i + 1))
        print("Average Accuracy of LSTM model on test : ", accuracy_test / (i + 1))
        print("Average F1 Score of LSTM model         :", f1_score / (i + 1))
        print()

In [194]:
# LSTM model
accuracy_average_test = 0
accuracy_average_train = 0
f1_score_average = 0

for i in range(iterations):
	model_lstm = Sequential([
		LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2])),
		# Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax'),
	])
 
	# save the architecture of the model
	if model_description_mode:
		print("LSTM")
		plot_model(model_lstm, colab_directory + 'model_lstm.png')
		print(model_lstm.summary())	
	else:
		model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
		model_lstm.fit(x_train, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
		# accuracy_average_test += model_lstm.evaluate(x_test, y_test, batch_size=batchSize, verbose=showWorking)[1]
		# accuracy_average_train += model_lstm.evaluate(x_train, y_train, batch_size=batchSize, verbose=showWorking)[1]

		# # f1-score
		# y_pred = model_lstm.predict_classes(x_test)
		# f1_score_average += sklearn.metrics.f1_score(y_test, y_pred, average='micro')
		accuracy_train, accuracy_test, f1_score = evaluate_dl_model(model_lstm, x_train, x_test, y_train, y_test)
		accuracy_average_train += accuracy_train
		accuracy_average_test += accuracy_test
		f1_score_average += f1_score

		if showWorkingAverage:
			print_dl_metrics(accuracy_average_train, accuracy_average_test, f1_score_average, i)
if not model_description_mode:
	print_dl_metrics(accuracy_average_train, accuracy_average_test, f1_score_average, i, average=True)
			# print(str(i + 1) + " train: " + str(accuracy_average_train / (i + 1)))
			# print(str(i + 1) + " test : " + str(accuracy_average_test / (i + 1)))
			# print(str(i + 1) + " f1   : " + str(f1_score_average / (i + 1)))
# if not model_description_mode:
# 	print_dl_metrics(accuracy_average_train, accuracy_average_test, f1_score_average, i, average=True)
	# accuracy_average_test /= iterations
	# accuracy_average_train /= iterations
	# f1_score_average /= iterations
	# print("Accuracy of LSTM model on train: ", accuracy_average_train)
	# print("Accuracy of LSTM model on test : ", accuracy_average_test)
	# print("F1 Score of LSTM model         :", f1_score_average)
# print()

1 train: 0.8964646386377739
1 test : 0.8799999952316284
1 f1   : 0.88

2 train: 0.8055555522441864
2 test : 0.755000003427267
2 f1   : 0.755

3 train: 0.8350168310030542
3 test : 0.7999999975164731
3 f1   : 0.7999999999999999

4 train: 0.8529040352112115
4 test : 0.827499995753169
4 f1   : 0.8275

5 train: 0.8313131271889717
5 test : 0.7999999973177909
5 f1   : 0.8

Average Accuracy of LSTM model on train:  0.8313131271889717
Average Accuracy of LSTM model on test :  0.7999999973177909
Average F1 Score of LSTM model         : 0.8



In [196]:
# CNN-LSTM
accuracy_average_train = 0
accuracy_average_test = 0
f1_score_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 20, x_test.shape[2]))

	# conv = Conv1D(filters=numFilters, kernel_size=3, activation=activationFunction)
	conv = Conv1D(filters=numFilters, kernel_size=3)
	# conv2 = Conv1D(filters=64, kernel_size=3, activation=activationFunction)
	model_lstm_cnn = Sequential([
		TimeDistributed(conv, input_shape=(None, 20, x_train.shape[2])),
		TimeDistributed(Activation(activation=activationFunction)),
		# TimeDistributed(conv2),
		# TimeDistributed(Dropout(dropoutRate)),
		TimeDistributed(MaxPooling1D(pool_size=2)),
		TimeDistributed(Flatten()),
		LSTM(200),
		# Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]), # y is one-hot encoded, hence its number of columns is number of classes
		Activation('softmax')
	])
 
	if model_description_mode:
		# save the architecture of the model
		print("CNN-LSTM")
		plot_model(model_lstm_cnn, colab_directory + 'model_lstm_cnn.png')
		print(model_lstm_cnn.summary())
	
	else:
		model_lstm_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
		model_lstm_cnn.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
		accuracy_train, accuracy_test, f1_score = evaluate_dl_model(model_lstm_cnn, trainX, testX, y_train, y_test)
		accuracy_average_train += accuracy_train
		accuracy_average_test += accuracy_test
		f1_score_average += f1_score

		if showWorkingAverage:
			print_dl_metrics(accuracy_average_train, accuracy_average_test, f1_score_average, i)
if not model_description_mode:
	print_dl_metrics(accuracy_average_train, accuracy_average_test, f1_score_average, i, average=True)

# 		accuracy_average_train += model_lstm_cnn.evaluate(trainX, y_train, batch_size=batchSize, verbose=showWorking)[1]
# 		accuracy_average_test += model_lstm_cnn.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
# 		if showWorkingAverage:
# 			print(str(i + 1) + " train: " + str(accuracy_average_train / (i + 1)))
# 			print(str(i + 1) + " test : " + str(accuracy_average_test / (i + 1)))

# if not model_description_mode:
# 	accuracy_average_test /= iterations
# 	accuracy_average_train /= iterations
# 	print("Accuracy of LSTM-CNN model on train: ", accuracy_average_train)
# 	print("Accuracy of LSTM-CNN model on test : ", accuracy_average_test)
# print()

1 train: 0.9191919092578117
1 test : 0.899999988079071
1 f1   : 0.9

2 train: 0.9154040295668322
2 test : 0.8899999916553497
2 f1   : 0.89

3 train: 0.9250841660531682
3 test : 0.9033333241939544
3 f1   : 0.9033333333333333

4 train: 0.9261363549816488
4 test : 0.9074999913573265
4 f1   : 0.9075

5 train: 0.9217171632882322
5 test : 0.9079999911785126
5 f1   : 0.908

Average Accuracy of LSTM model on train:  0.9217171632882322
Average Accuracy of LSTM model on test :  0.9079999911785126
Average F1 Score of LSTM model         : 0.908



In [197]:
# LSTM + conv
accuracy_average_train = 0
accuracy_average_test = 0
f1_score_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 1, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 1, 20, x_test.shape[2]))

	model_lstm_conv = Sequential([
		ConvLSTM2D(filters=numFilters, kernel_size=(1,3), input_shape=(3, 1, 20, x_train.shape[2])),
		Activation(activationFunction),
		# Dropout(dropoutRate),
		Flatten(),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax')
	])
 
	if model_description_mode:
		print("Convolutional LSTM")
		# save the architecture of the model
		plot_model(model_lstm_conv, colab_directory + 'model_lstm_conv.png')
		print(model_lstm_conv.summary())
	
	else: 
		model_lstm_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
		model_lstm_conv.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
		accuracy_train, accuracy_test, f1_score = evaluate_dl_model(model_lstm_conv, trainX, testX, y_train, y_test)
		accuracy_average_train += accuracy_train
		accuracy_average_test += accuracy_test
		f1_score_average += f1_score

		if showWorkingAverage:
			print_dl_metrics(accuracy_average_train, accuracy_average_test, f1_score_average, i)
if not model_description_mode:
	print_dl_metrics(accuracy_average_train, accuracy_average_test, f1_score_average, i, average=True)
# 		accuracy_average_test += model_lstm_conv.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
# 		accuracy_average_train += model_lstm_conv.evaluate(trainX, y_train, batch_size=batchSize, verbose=showWorking)[1]
# 		if showWorkingAverage:
# 			print(str(i + 1) + " train: " + str(accuracy_average_train / (i + 1)))
# 			print(str(i + 1) + " test : " + str(accuracy_average_test / (i + 1)))
# if not model_description_mode:
# 	accuracy_average_test /= iterations
# 	accuracy_average_train /= iterations
# 	print("Accuracy of LSTM-Conv model on train: ", accuracy_average_train)
# 	print("Accuracy of LSTM-Conv model on test : ", accuracy_average_test)
# print()

1 train: 0.9166666591408277
1 test : 0.8899999976158142
1 f1   : 0.89

2 train: 0.9280302959560144
2 test : 0.909999993443489
2 f1   : 0.91

3 train: 0.9318181748944099
3 test : 0.9166666567325592
3 f1   : 0.9166666666666666

4 train: 0.9337121139873157
4 test : 0.9199999898672104
4 f1   : 0.92

5 train: 0.9328282755432706
5 test : 0.921999990940094
5 f1   : 0.922

Average Accuracy of LSTM model on train:  0.9328282755432706
Average Accuracy of LSTM model on test :  0.921999990940094
Average F1 Score of LSTM model         : 0.922



In [110]:
# SVM

X = np.load(colab_directory + 'x_svm.npy')
Y = np.load(colab_directory + 'y_svm.npy')
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=2)
clf = SVC(gamma='auto', kernel='rbf', verbose=True, probability=True)
clf.fit(x_train, y_train)
print("Train accuracy: " + str(clf.score(x_train, y_train)))
y_pred = clf.predict(x_test)
print("Test accuracy: " + str(clf.score(x_test, y_test)))

[LibSVM]Train accuracy: 0.6596742349457059
Test accuracy: 0.6653504442250741


In [0]:
# f-score

# svm
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=2) # fix it later
# y_pred = clf.predict(x_test)
# f_score_svm = sklearn.metrics.f1_score(y_test, y_pred, average='micro')
# print(f_score_svm)

# lstm
# x = np.load(colab_directory + "x.npy")
# y = np.load(colab_directory + "y.npy")
# x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
# trainX = x_train.reshape((x_train.shape[0], 3, 20, x_train.shape[2]))
# testX = x_test.reshape((x_test.shape[0], 3, 20, x_test.shape[2]))
# y_pred = model_lstm_cnn.predict_classes(testX)
# f_score_lstm = sklearn.metrics.f1_score(y_test, y_pred, average='micro')
# print(f_score_lstm)
# print(y_pred)
# print(y_pred.shape)



In [206]:
# saving data
# np.save(colab_directory + "x_train_lstm", x_train)
# np.save(colab_directory + "x_test_lstm", x_test)
# np.save(colab_directory + "y_train_lstm", y_train)
# np.save(colab_directory + "y_test_lstm", y_test)
# print(x_train.shape)

# saving model
# model_lstm.save(colab_directory + "model_lstm.h5")
# model_lstm_cnn.save(colab_directory + "model_lstm_cnn.h5")
# model_lstm_conv.save(colab_directory + "model_lstm_conv.h5")

# np.save(colab_directory + "y_train_classes_lstm", y_train_classes)
# np.save(colab_directory + "y_test_classes_lstm", y_test_classes)
# np.save(colab_directory + "x_train_svm")
# dump(clf, colab_directory + "svm.joblib")



['/content/drive/My Drive/Colab Notebooks/ML Course Project/svm.joblib']